In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
pd.options.mode.chained_assignment = None  # default='warn'

### Data processing

In [2]:
# space track gp data
df1 = pd.read_csv('space-track.gp.1206.csv')
gp = df1[['NORAD_CAT_ID','OBJECT_NAME','OBJECT_ID','LAUNCH_DATE','DECAY_DATE','CREATION_DATE','EPOCH','OBJECT_TYPE','APOAPSIS','PERIAPSIS','COUNTRY_CODE','MEAN_MOTION', 'ECCENTRICITY','PERIOD']]
gp['LAUNCH_DATE'] = pd.to_datetime(gp['LAUNCH_DATE'], format='%Y-%m-%d')
gp["Year_launch"] = gp['LAUNCH_DATE'].dt.year
gp['DECAY_DATE'] = pd.to_datetime(gp['DECAY_DATE'], format='%Y-%m-%d')
gp["Year_decay"] = gp['DECAY_DATE'].dt.year
gp = gp.fillna({'Year_decay':2020})

# data from PDF
df2 = pd.read_csv("SATELLITE_BREAKUPS.csv")
df2[["LAUNCH DATE", "BREAKUP DATE"]] = df2[["LAUNCH DATE", "BREAKUP DATE"]].apply(pd.to_datetime)
df2["launch_designator"] = df2["INTERNATIONAL DESIGNATOR"].str.slice(stop=8)
df2 = df2.drop_duplicates(subset="launch_designator")

# space track satcat data
df3 = pd.read_csv("satcat_all.csv")
df3[["DEBUT", "DECAY", "LAUNCH"]] = df3[["DEBUT", "DECAY", "LAUNCH"]].apply(pd.to_datetime)
df3["launch_designator"] = df3.INTLDES.str.slice(stop=8)

# combine df3 and df2
combined = df3.merge(df2[["launch_designator","BREAKUP DATE"]], how="left", left_on="launch_designator", right_on="launch_designator")
combined = combined[['INTLDES', 'NORAD_CAT_ID', 'OBJECT_TYPE', 'SATNAME', 'DEBUT', 'COUNTRY','LAUNCH', 'DECAY','RCS_SIZE','LAUNCH_YEAR','LAUNCH_NUM', 'LAUNCH_PIECE', 'OBJECT_NAME', 'OBJECT_ID','OBJECT_NUMBER', 'launch_designator', 'BREAKUP DATE']]

C:\Users\cmial\Anaconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


All relevant data is now combined int he `combined` dataframe.  We just need to select the correct date to use with the following rules:
1. For non-DEBRIS, we can use the LAUNCH
1. For debris listed in the PDF file, we should use `BREAKUP DATE`
1. If the `DEBUT` is earlier than `DECAY`, we can probably trust it
1. If it `DEBUT` later than 2005, we can also trust it since these are reported to space track as soon as they are spotted
1. if a debris `DECAYED` within 1 year of `LAUNCH`, let's just assumed that it was created on launch

In [3]:
# For non-DEBRIS, we can use the LAUNCH
combined.loc[(combined.OBJECT_TYPE!="DEBRIS"),"exist_date"] = combined.LAUNCH

# For debris listed in the PDF file, we should use BREAKUP DATE
combined.loc[((combined.OBJECT_TYPE=="DEBRIS") & combined["BREAKUP DATE"].notnull()),"exist_date"] = combined["BREAKUP DATE"]

# For the remaining debris without dates, if the `DEBUT` is earlier than `DECAY`, we use debut date
combined.loc[((combined.OBJECT_TYPE=="DEBRIS") & combined["exist_date"].isnull() & (combined.DECAY > combined.DEBUT)),"exist_date"] = combined.DEBUT

# If it `DEBUT` later than 2005, we can also trust it since these are reported to space track as soon as they are spotted
combined.loc[((combined.OBJECT_TYPE=="DEBRIS") & combined["exist_date"].isnull() & (combined.DEBUT > "2005")),"exist_date"] = combined.DEBUT

# if a debris DECAYED within 1 year of launch, let's just assumed that it was created on launch
combined.loc[(combined["exist_date"].isnull() & (combined.DECAY - combined.LAUNCH < datetime.timedelta(days=365))),"exist_date"] = combined.LAUNCH

# set the rest to launch date since we don't have anything else to go by
combined.loc[(combined["exist_date"].isnull()),"exist_date"] = combined.LAUNCH

In [4]:
# count the total number of objects per object type per year
gb = pd.DataFrame(combined.groupby(['OBJECT_TYPE', 'exist_date'])['INTLDES'].count())
gb.columns = ['Count']
gb.reset_index(inplace=True)
gb['year'] = gb['exist_date'].dt.year
gb = gb[gb['OBJECT_TYPE']!='TBA']
gb['OBJECT_TYPE'] = gb['OBJECT_TYPE'].apply(lambda x: 'DEBRIS' if x=='ROCKET BODY' else x)
gb1 = pd.DataFrame(gb.groupby(['OBJECT_TYPE', 'year'])['Count'].sum()).groupby(level=0).cumsum().reset_index()

In [5]:
# count the number of decay per object per year
gb_decay = pd.DataFrame(combined.groupby(['OBJECT_TYPE', 'DECAY'])['INTLDES'].count())
gb_decay.columns = ['Count']
gb_decay.reset_index(inplace=True)
gb_decay['year'] = gb_decay['DECAY'].dt.year
gb_decay = gb_decay[gb_decay['OBJECT_TYPE']!='TBA']
gb_decay['OBJECT_TYPE'] = gb_decay['OBJECT_TYPE'].apply(lambda x: 'DEBRIS' if x=='ROCKET BODY' else x)
gb_decay1 = pd.DataFrame(gb_decay.groupby(['OBJECT_TYPE', 'year'])['Count'].sum()).groupby(level=0).cumsum().reset_index()

In [7]:
obj_count = pd.merge(gb1, gb_decay1, on=['year','OBJECT_TYPE'], how='outer')
obj_count['Count_y'] = obj_count['Count_y'] .fillna(0)
obj_count['Count_x'] = obj_count['Count_x'] .fillna(0)
obj_count['live'] = obj_count['Count_x'] - obj_count['Count_y']
obj_count.rename(columns={'Count_x': 'Total', 'Count_y':'Decay'},inplace=True)

### First let's take a look at the number of space objects (Debris, Rocket Body, Payload) from year 1958 to 2020
We observe that the number of all three types of objects has increased in the past 60 years especially for debris. Two major events happened in 2007 and 2009 causing a significant increase of debris. Debris appears to decay the fastest while rocket body decays the slowest.

In [22]:
def plot_ts(live, decay, live_col, p1, p2, ax, ay):
    '''
    Plot time series data: number of debris and payload 
    from 1958 to 2020.
    inputs: live (df), decay(df), 
    live_col: string
    y1, y2, ax, ay: position of the annotations and arrows
 
    '''
    x1 = live[live['OBJECT_TYPE']=='DEBRIS']['year']
    y1 = live[live['OBJECT_TYPE']=='DEBRIS'][live_col]
    x2 = live[live['OBJECT_TYPE']=='PAYLOAD']['year']
    y2 = live[live['OBJECT_TYPE']=='PAYLOAD'][live_col]
    x3 = decay[decay['OBJECT_TYPE']=='DEBRIS']['year']
    y3 = decay[decay['OBJECT_TYPE']=='DEBRIS']['Count']
    x4 = decay[decay['OBJECT_TYPE']=='PAYLOAD']['year']
    y4 = decay[decay['OBJECT_TYPE']=='PAYLOAD']['Count']
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x1, y=y1,name='Debris', line=dict(color='crimson')))
    fig.add_trace(go.Scatter(x=x3, y=y3,name='Debris Decay', line=dict(color='crimson', dash='dot')))
    fig.add_trace(go.Scatter(x=x2, y=y2,name='Payload', line=dict(color='dodgerblue')))
    fig.add_trace(go.Scatter(x=x4, y=y4,name='Payload Decay', line=dict(color='dodgerblue', dash='dot')))
    fig.add_annotation(x=2007, y=p1, text='2007:Destruction of'+ '<br>'+ 'Chinese satellite by missile',showarrow=True,arrowhead=3,ax=ax, ay=ay,xanchor='right',yanchor='bottom')
    fig.add_annotation(x=2009 , y=p2, text='2009:Accidental collision'+ '<br>'+ 'of US and Russian satellites',showarrow=True,arrowhead=2,xanchor='right',yanchor='bottom')
    x_ticks = list(np.arange(1958, 2020, 3))
    fig.update_layout(xaxis = dict(tickmode = 'array', tickvals = x_ticks), xaxis_range=[1958,2020],
                      title={'text':'Number of Space Objects','x':0.4,'y':0.85}, 
                             xaxis_title='Year', yaxis_title = "Number of items")

    fig.show()
    

In [23]:
plot_ts(obj_count, gb_decay1,'live', 12000, 14500, -60, 10)

In [24]:
plot_ts(gb1, gb_decay1, 'Count', 29000, 32000, -20, 30)

### Look at the number of space objects from a different perspective - Satellite Orbits
There are significantly more objects in the Lower Earth Orbit (LEO) than in other orbits. We will focus on the activities in LEO for further analysis.

In [25]:
# combine with gp data to get infor such as APOAPSIS, PERIAPSIS, inclination that are not available in combined data
df4 = pd.merge(combined, gp, how='right', left_on="INTLDES", right_on="OBJECT_ID" )
df4 = df4[df4['exist_date'].notnull()]
df4 = df4.drop(columns=['OBJECT_TYPE_x', 'NORAD_CAT_ID_x','OBJECT_NAME_x','OBJECT_ID_x'])
df4.rename(columns={'NORAD_CAT_ID_y': 'NORAD_CAT_ID','OBJECT_NAME_y': 'OBJECT_NAME','OBJECT_ID_y':'OBJECT_ID', 'OBJECT_TYPE_y':'OBJECT_TYPE'}, inplace=True)
#select columns needed 
df4 = df4[['INTLDES', 'SATNAME', 'COUNTRY','exist_date','OBJECT_TYPE', 'APOAPSIS', 'PERIAPSIS','MEAN_MOTION', 'ECCENTRICITY','PERIOD']]

In [26]:
# for 1025 items not having orbit classification based on space track's definition of orbits
# additional rules:
# LEO: periapsis < 2000 km
# MEO : 2000 km < periapsis < 35786 km
# HEO : 35786 km < periapsis 

def categorize(row):
    if (row['MEAN_MOTION']>11.25) & (row['ECCENTRICITY']<0.25):
        return "LEO"
    elif (row['PERIOD']>=600) & (row['PERIOD']<=800) & (row['ECCENTRICITY']<0.25):
        return "MEO"
    elif (row['MEAN_MOTION']>=0.99) & (row['MEAN_MOTION']<=1.01) & (row['ECCENTRICITY']<0.01):
        return "GEO"
    elif row['ECCENTRICITY'] > 0.25:
        return "HEO"
    else:
        if row['PERIAPSIS'] < 2000:
            return "LEO"
        if (row['PERIAPSIS'] < 35786) & (row['PERIAPSIS'] > 2000):
            return "MEO"

In [27]:
df4['Orbit'] = df4.apply(categorize, axis=1)
df4['year_exist'] = df4['exist_date'].dt.year
orbit_type = pd.DataFrame(df4.groupby(['Orbit','year_exist'])['INTLDES'].count()).groupby(level=0).cumsum().reset_index()
orbit_type.rename(columns={'INTLDES':'count'}, inplace=True)

In [29]:
x_ticks = list(np.arange(1958, 2020, 3))
fig = px.bar(orbit_type, x='year_exist', y='count', color='Orbit', 
             labels={'year_exist': 'Year', "count":"Number of Ojects"})
fig.update_layout(title={'text': "Number of Space Objects in Different Orbits",
                         'x':0.5, 'y':0.92}, barmode="stack",
                 xaxis = dict(tickmode = 'array', tickvals = x_ticks))
fig.show()

#### *Here, we can use Nick's LEO density plot to illustrate what we find after delving into LEO.*

### Visualize the two events mentioned earlier using Gabbard graph

In [11]:
# fengyun debris
fy = gp[(gp['OBJECT_TYPE']=='DEBRIS') & (gp['OBJECT_NAME'].str.contains('FENGYUN')) & (gp['DECAY_DATE'].notnull())]

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=fy['PERIOD'], y=fy['APOAPSIS'], mode='markers', name='Apogee'))
fig.add_trace(go.Scatter(x=fy['PERIOD'], y=fy['PERIAPSIS'], mode='markers', name='Perigee'))
fig.update_layout(title={'text':'Gabbard Plot of FengYun Debris','x':0.45,'y':0.85}, 
                         xaxis_title='Period(min)', yaxis_title = "Altitude(km)")
fig.show()

In [32]:
# Iridium 33 (commerical) and Kosmos-2251 (miliatry)
ik = gp[(gp['OBJECT_TYPE']=='DEBRIS') & (gp['OBJECT_NAME'].str.contains('IRIDIUM 33') 
            | (gp['OBJECT_NAME'].str.contains('COSMOS 2251')))]
ik = ik[(ik['PERIOD']<150) & (ik['PERIAPSIS']>0)]

In [33]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ik.loc[ik['OBJECT_NAME']=='COSMOS 2251 DEB','PERIOD'], y=ik.loc[ik['OBJECT_NAME']=='COSMOS 2251 DEB','APOAPSIS'], mode='markers', name='Apogee COSMOS'))
fig.add_trace(go.Scatter(x=ik.loc[ik['OBJECT_NAME']=='COSMOS 2251 DEB','PERIOD'], y=ik.loc[ik['OBJECT_NAME']=='COSMOS 2251 DEB','PERIAPSIS'], mode='markers', name='Perigee COSMOS'))

fig.add_trace(go.Scatter(x=ik.loc[ik['OBJECT_NAME']=='IRIDIUM 33 DEB','PERIOD'], y=ik.loc[ik['OBJECT_NAME']=='IRIDIUM 33 DEB','APOAPSIS'], mode='markers', name='Apogee IRIDIUM'))
fig.add_trace(go.Scatter(x=ik.loc[ik['OBJECT_NAME']=='IRIDIUM 33 DEB','PERIOD'], y=ik.loc[ik['OBJECT_NAME']=='IRIDIUM 33 DEB','PERIAPSIS'], mode='markers', name='Perigee IRIDIUM'))

fig.update_layout(title={'text':'Gabbard Plot of COSMOS and IRIDIUM Collision','x':0.45,'y':0.85}, 
                         xaxis_title='Period(min)', yaxis_title = "Altitude(km)")
fig.show()